# Connecting to the database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Data overview

In [14]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The Chinook record store has just signed a deal with a new record label, and you need to select the first three albums to add to the store, from a list of four. All four albums are by artists who don't have any tracks in the store right now — we have the artists' names and the genre of music they produce:

Artist Name / Genre

Regal / Hip-Hop

Red Tone / Punk

Meteor and the Girls / Pop

Slim Jim Bites / Blues

Write a query that returns each genre with the number of tracks sold in the USA:
- in absolute numbers
- in percentages

Write a paragraph that interprets the data and makes a recommendation for the three artists whose albums we should purchase for the store, based on sales of tracks from their genres.

In [18]:
#We need to JOIN all the information we need from the various tables.

In [7]:
%%sql

WITH usa_sales AS
    (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i ON il.invoice_id = i.invoice_id
    INNER JOIN customer c ON i.customer_id = c.customer_id
    WHERE c.country = "USA"
    )
SELECT *
FROM usa_sales us
INNER JOIN track t on t.track_id = us.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
LIMIT 10;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1,genre_id_1,name_1
1,1,1158,0.99,1,1158,Right Next Door to Hell,91,2,1,None,182321,3175950,0.99,1,Rock
2,1,1159,0.99,1,1159,Dust N' Bones,91,2,1,None,298374,5053742,0.99,1,Rock
3,1,1160,0.99,1,1160,Live and Let Die,91,2,1,None,184016,3203390,0.99,1,Rock
4,1,1161,0.99,1,1161,Don't Cry (Original),91,2,1,None,284744,4833259,0.99,1,Rock
5,1,1162,0.99,1,1162,Perfect Crime,91,2,1,None,143637,2550030,0.99,1,Rock
6,1,1163,0.99,1,1163,You Ain't the First,91,2,1,None,156268,2754414,0.99,1,Rock
7,1,1164,0.99,1,1164,Bad Obsession,91,2,1,None,328282,5537678,0.99,1,Rock
8,1,1165,0.99,1,1165,Back off Bitch,91,2,1,None,303436,5135662,0.99,1,Rock
9,1,1166,0.99,1,1166,Double Talkin' Jive,91,2,1,None,203637,3520862,0.99,1,Rock
10,1,1167,0.99,1,1167,November Rain,91,2,1,None,537540,8923566,0.99,1,Rock


In [3]:
#We can make the entire query in only one step (as shown below) but is always important to
#have a look at the data and understand well how the different tables look like, hence why
#the previous step. 

In [13]:
%%sql

WITH usa_sales AS
    (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i ON il.invoice_id = i.invoice_id
    INNER JOIN customer c ON i.customer_id = c.customer_id
    WHERE c.country = "USA"
    )
SELECT
    g.name genre,
    COUNT(us.invoice_line_id) tracks_sold,
    CAST(COUNT(us.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_sales) percentage_sold
FROM usa_sales us
INNER JOIN track t on t.track_id = us.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the sales, the 3 albums we should recommend are: 

Red Tone / Punk (12%)

Slim Jim Bites / Blues (3%)

Meteor and the Girls / Pop (2%)

There is a clear difference between 'Rock' and any other genre, with 53% and trending, current and upcoming rock albums should be looked at with major importance. 

**Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you think are relevant to the analysis.**

**Write a short statement describing your results, and providing a possible interpretation.**

In [42]:
%%sql

WITH employee_sales AS
    (
    SELECT 
        i.customer_id,
        c.support_rep_id,
        SUM(i.total) total
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY 2
    )
    
SELECT
    e.first_name || " " || e.last_name employee,
    SUM(i.total) total_sales,
    e.hire_date
FROM employee_sales es
INNER JOIN employee e ON e.employee_id = es.support_rep_id
GROUP BY 1;

Done.


employee,total_sales,hire_date
Jane Peacock,1731.510000000004,2017-04-01 00:00:00
Margaret Park,1584.0000000000032,2017-05-03 00:00:00
Steve Johnson,1393.9200000000028,2017-10-17 00:00:00


In [3]:
#Jane vs Steve

In [2]:
1393.92/1731.51

0.8050314465408805

Steve has sold around 20% less than Jane.

The difference in sales per employee can be explained due to the fact that their hiring dates are completely different. As expected, Jane, who has been in the company the longest is the highest seller. On the other hand, Steve, who joined 6 months later has the lowest total sales, most likely due to the fact that he join much later.

As the employees started in completely different dates, it would be interesting to understand how the employees are performing in terms of revenue per customer.

In [11]:
%%sql    
    SELECT 
            e.first_name || ' ' || e.last_name employee_name,
            SUM(i.total) total_sales,
            count(distinct c.customer_id) number_of_customer_assigned,
            SUM(i.total) / count(distinct c.customer_id) avg_sales_per_customer
    FROM invoice i  
         INNER JOIN customer c ON c.customer_id = i.customer_id
         INNER JOIN employee e ON e.employee_id = c.support_rep_id
    GROUP BY 1;

Done.


employee_name,total_sales,number_of_customer_assigned,avg_sales_per_customer
Jane Peacock,1731.5100000000039,21,82.45285714285733
Margaret Park,1584.0000000000034,20,79.20000000000017
Steve Johnson,1393.920000000002,18,77.44000000000011


Although Steve was hired 6 months before Jane, we can see that they've dealt with almost the same amount of distinct customers. Due to the difference in total sales, this might mean that Jane was assigned the most valuable customers perhaps due to being more experienced, we can't be sure as we are not aware of the criteria on how customers are assigned to sales reps at Chinook.

**Write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.
The results should sort by the total sales from highest to lowest, with the "Other" group at the very bottom.**

**For each country, include the following:
total number of customers
total value of sales
average value of sales per customer
average order value**

In [4]:
%%sql

CREATE VIEW all_countries AS
     SELECT country,
            CASE
                WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN "Other"
                ELSE country
            END AS country_grp,            
            COUNT(DISTINCT(c.customer_id)) AS number_of_customers,
            COUNT(DISTINCT(i.invoice_id)) AS number_of_orders,
            SUM(i.total) AS total_value_of_sales,
            CASE
                WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 0
                ELSE 1
            END AS sort
FROM customer AS c
INNER JOIN invoice AS i ON c.customer_id = i.customer_id
GROUP BY country;

Done.


[]

In [5]:
%%sql

SELECT * FROM all_countries LIMIT 10;

Done.


country,country_grp,number_of_customers,number_of_orders,total_value_of_sales,sort
Argentina,Other,1,5,39.6,0
Australia,Other,1,10,81.18,0
Austria,Other,1,9,69.3,0
Belgium,Other,1,7,60.38999999999999,0
Brazil,Brazil,5,61,427.68000000000006,1
Canada,Canada,8,76,535.5900000000001,1
Chile,Other,1,13,97.02000000000001,0
Czech Republic,Czech Republic,2,30,273.24000000000007,1
Denmark,Other,1,10,37.61999999999999,0
Finland,Other,1,11,79.2,0


In [78]:
%%sql

SELECT  country_grp AS country,
        SUM(number_of_customers) AS total_number_of_customers,
        ROUND(SUM(total_value_of_sales), 2) AS total_value_of_sales,
        ROUND((SUM(total_value_of_sales) / SUM(number_of_customers)), 2) AS average_value_of_sales_per_customer,
        ROUND((SUM(total_value_of_sales) / SUM(number_of_orders)), 2) AS average_order_value        
  FROM  all_countries
 GROUP  BY country_grp
 ORDER  BY sort DESC, total_value_of_sales DESC;

Done.


country,total_number_of_customers,total_value_of_sales,average_value_of_sales_per_customer,average_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


USA (1041), Canada (536) and Brazil (428) account for the top 3 in terms of total value of sales. 

The country with the highest average sales per customer is the Czech Republic (137) but with only 2 customers in total. From the top 3 countries in terms of total sales, Brazil has the highest average sale per customer with 86.

It's also worthwhile mentioning USA is the top country in terms of total customers (13) and the Czech Republic has the highest average order value (9.11).